In [35]:
# ============================================================
# CIO Momentum Report (Equities, USD) — PDF Pro
# ============================================================

import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
matplotlib.use("Agg")  # rendu off-screen (utile si script)
import matplotlib.pyplot as plt
from io import BytesIO
from datetime import datetime, timedelta

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table
from reportlab.platypus import TableStyle
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

# ---------- Paramètres ----------
OUTPUT_PDF = "Rapport_Momentum.pdf"
LOOKBACK_YEARS = 4  # on télécharge 4 ans pour être à l’aise sur 3Y
HEATMAP_DPI = 180

# ---------- Indices & FX (conversion en USD) ----------
tickers = {
    "US": "^GSPC",        # S&P 500 (USD)
    "Europe": "^STOXX50E",# EuroStoxx 50 (EUR)
    "Japan": "^N225",     # Nikkei 225 (JPY)
    "China": "^HSI",      # Hang Seng (HKD)
    "LatAm": "^BVSP"      # Bovespa (BRL)
}
fx_map = {
    "US": None,           # déjà USD
    "Europe": "EURUSD=X",
    "Japan": "JPYUSD=X",
    "China": "HKDUSD=X",
    "LatAm": "BRLUSD=X"
}

# ---------- Styles ReportLab (tables finance) ----------
table_style = TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#2F4F4F")),  # header gris foncé
    ('TEXTCOLOR',(0,0),(-1,0),colors.white),                    # texte header blanc
    ('ALIGN',(1,1),(-1,-1),'RIGHT'),                            # chiffres à droite
    ('ALIGN',(0,0),(0,-1),'LEFT'),                              # 1ère colonne à gauche
    ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),              # header bold
    ('FONTSIZE', (0,0), (-1,-1), 9),                            # taille police
    ('BOTTOMPADDING', (0,0), (-1,0), 6),
    ('BACKGROUND',(0,1),(-1,-1),colors.whitesmoke),             # fond data gris clair
    ('GRID', (0,0), (-1,-1), 0.25, colors.grey)                 # grille fine
])

# ---------- Fonctions utilitaires ----------
def month_end_prices(px: pd.DataFrame) -> pd.DataFrame:
    """Dernier cours de chaque mois."""
    return px.resample("M").last()

def perf_point(df_monthly: pd.DataFrame, months: int) -> pd.Series:
    """Perf cumulée sur N mois à la dernière date disponible."""
    return df_monthly.pct_change(periods=months).iloc[-1]

def make_perf_table(perfs: pd.DataFrame) -> list:
    """Formate le tableau de performances (%) pour ReportLab."""
    header = [""] + list(perfs.columns)
    rows = []
    for idx in perfs.index:
        row = [idx] + [f"{100*perfs.loc[idx, col]:.2f}" for col in perfs.columns]
        rows.append(row)
    return [header] + rows

def make_alloc_table(weights: pd.Series, title_cols=("Région","Poids %")) -> list:
    """Formate le tableau d’allocations (%) trié décroissant."""
    w = (weights.sort_values(ascending=False) * 100).round(1)
    data = [list(title_cols)] + [[idx, f"{val:.1f}"] for idx, val in w.items()]
    return data

# ---------- Téléchargement des données ----------
start_date = (pd.Timestamp.today().normalize() - pd.DateOffset(years=LOOKBACK_YEARS)).strftime("%Y-%m-%d")

# Prix indices (Close)
raw = yf.download(list(tickers.values()), start=start_date, progress=False)
if "Close" in raw:
    prices = raw["Close"].copy()
else:
    # fallback si yfinance renvoie directement un DF simple
    prices = raw.copy()

# FX vers USD (Close)
fx_list = [fx for fx in fx_map.values() if fx is not None]
fx_prices = yf.download(fx_list, start=start_date, progress=False)["Close"] if fx_list else pd.DataFrame()

# ---------- Conversion en USD ----------
prices_usd = pd.DataFrame(index=prices.index)
for region, idx in tickers.items():
    p = prices[idx].copy()
    if fx_map[region] is None:
        prices_usd[region] = p  # déjà USD
    else:
        rate = fx_prices[fx_map[region]].reindex(p.index).fillna(method="ffill")
        prices_usd[region] = p * rate  # converti en USD

# ---------- Données mensuelles et perfs multi-horizons ----------
monthly = month_end_prices(prices_usd).dropna(how="any")
horizons = {"1M":1, "3M":3, "6M":6, "1Y":12, "3Y":36}

# Vérifier qu’on a assez d’historique pour 3Y
if len(monthly) < (max(horizons.values()) + 1):
    raise ValueError("Pas assez d’historique pour calculer 3Y. Augmente LOOKBACK_YEARS ou change les horizons.")

perfs = pd.DataFrame({h: perf_point(monthly, m) for h, m in horizons.items()})
perfs = perfs.reindex(columns=["1M","3M","6M","1Y","3Y"])  # ordre lisible
ranks = perfs.rank(ascending=False)  # 1 = meilleur

# ---------- Allocations (Momentum & Vol-adjusted) ----------
# Score momentum = somme des points (1->5 pts ... 5->1)
points = (6 - ranks).sum(axis=1)   # somme des points sur tous les horizons
weights = (points / points.sum()).rename("Momentum Weights")

# Vol annualisée (à partir des retours mensuels) — fenêtre 6M
monthly_returns = monthly.pct_change()
vol_6m = monthly_returns.rolling(window=6).std().iloc[-1] * np.sqrt(12)
# Ajustement par le risque
adj_scores = points / vol_6m
adj_weights = (adj_scores / adj_scores.sum()).rename("VolAdj Weights")

# ---------- Commentaire automatique (CIO-style bref) ----------
leaders = weights.sort_values(ascending=False).head(2).index.tolist()
laggard = weights.sort_values().head(1).index[0]
comment = f"OW {leaders[0]} & {leaders[1]}, UW {laggard}."

# ---------- Heatmap des rangs (chiffres visibles) ----------
ranks_int = ranks.astype(int)
plt.figure(figsize=(8, 6))
ax = sns.heatmap(
    ranks_int,
    annot=True, fmt="d",
    linewidths=0.5, linecolor="grey",
    cmap="RdYlGn_r",
    cbar=True, cbar_kws={'label': 'Rang (1 = Top performer)'},
    annot_kws={"size":12, "weight":"bold", "color":"black"}
)
plt.title("Momentum Ranking des Indices (en USD, 1 = Top)", fontsize=14, weight="bold", pad=15)
plt.ylabel("Indices", fontsize=12, weight="bold")
plt.xlabel("Horizon", fontsize=12, weight="bold")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(rotation=0, fontsize=11)
plt.tight_layout()

heatmap_buf = BytesIO()
plt.savefig(heatmap_buf, format="png", bbox_inches="tight", dpi=HEATMAP_DPI)
plt.close()
heatmap_buf.seek(0)

# ---------- Construction du PDF ----------
styles = getSampleStyleSheet()
doc = SimpleDocTemplate(OUTPUT_PDF, pagesize=A4)
story = []

# Titre + date
today_str = pd.Timestamp.today().strftime("%d %B %Y")
story.append(Paragraph("📌 CIO Momentum Report (Equities, USD)", styles['Title']))
story.append(Spacer(1, 6))
story.append(Paragraph(f"<i>Rapport généré le {today_str}</i>", styles['Normal']))
story.append(Spacer(1, 12))

# Commentaire
story.append(Paragraph(f"<b>Commentaire :</b> {comment}", styles['Normal']))
story.append(Spacer(1, 12))

# Heatmap
story.append(Paragraph("<b>Heatmap des Rangs Momentum</b>", styles['Heading3']))
story.append(Image(heatmap_buf, width=400, height=250))
story.append(Spacer(1, 12))

# ---------- Graphique YTD base 100 (fin de mois) ----------
ytd_start = pd.Timestamp(datetime.today().year, 1, 1)

# Données de fin de mois depuis début d'année
monthly_ytd = monthly[monthly.index >= ytd_start].copy()

# Rebase à 100 sur la première valeur non-NaN de chaque série
ytd_rebased = monthly_ytd.apply(lambda x: x / x.dropna().iloc[0] * 100)

plt.figure(figsize=(8,5))
for col in ytd_rebased.columns:
    plt.plot(ytd_rebased.index, ytd_rebased[col], label=col, linewidth=2)


plt.title("Indices mondiaux — Performance YTD (Base 100, en USD, fin de mois)", fontsize=14, weight="bold")
plt.ylabel("Base 100")
plt.xlabel("Date")
plt.legend(loc="upper left", fontsize=9)
plt.grid(True, linestyle="--", alpha=0.6)

ytd_buf = BytesIO()
plt.savefig(ytd_buf, format="png", bbox_inches="tight", dpi=150)
plt.close()
ytd_buf.seek(0)

story.append(Paragraph("<b>Évolution YTD des indices (Base 100 en USD, fin de mois)</b>", styles['Heading3']))
story.append(Image(ytd_buf, width=400, height=250))
story.append(Spacer(1, 12))


# ---------- Graphique 3 ans base 100 (fin de mois) ----------
three_year_start = pd.Timestamp.today() - pd.DateOffset(years=3)

monthly_3y = monthly[monthly.index >= three_year_start].copy()
px_3y_rebased = monthly_3y.apply(lambda x: x / x.dropna().iloc[0] * 100)

plt.figure(figsize=(8,5))
for col in px_3y_rebased.columns:
    plt.plot(px_3y_rebased.index, px_3y_rebased[col], label=col, linewidth=2)

plt.title("Indices mondiaux — Performance 3 ans (Base 100, en USD, fin de mois)", fontsize=14, weight="bold")
plt.ylabel("Base 100")
plt.xlabel("Date")
plt.legend(loc="upper left", fontsize=9)
plt.grid(True, linestyle="--", alpha=0.6)

three_buf = BytesIO()
plt.savefig(three_buf, format="png", bbox_inches="tight", dpi=150)
plt.close()
three_buf.seek(0)

story.append(Paragraph("<b>Évolution 3 ans des indices (Base 100 en USD, fin de mois)</b>", styles['Heading3']))
story.append(Image(three_buf, width=400, height=250))
story.append(Spacer(1, 12))

# Tableau perfs
story.append(Paragraph("<b>Performance (%) en USD</b>", styles['Heading3']))
perf_table = make_perf_table(perfs)
t_perf = Table(perf_table)
t_perf.setStyle(table_style)
story.append(t_perf)
story.append(Spacer(1, 12))

# Allocation recommandée (momentum)
story.append(Paragraph("<b>Allocation recommandée — Momentum (%)</b>", styles['Heading3']))
alloc_table = make_alloc_table(weights, title_cols=("Région","Poids %"))
t_alloc = Table(alloc_table)
t_alloc.setStyle(table_style)
story.append(t_alloc)
story.append(Spacer(1, 12))

# Allocation ajustée par volatilité
story.append(Paragraph("<b>Allocation ajustée par volatilité (%)</b>", styles['Heading3']))
alloc_vol_table = make_alloc_table(adj_weights, title_cols=("Région","Poids %"))
t_alloc_vol = Table(alloc_vol_table)
t_alloc_vol.setStyle(table_style)
story.append(t_alloc_vol)
story.append(Spacer(1, 12))

# Note de pied (transparence méthodo)
story.append(Paragraph(
    "<font size=8><i>Notes: Prix d'indices convertis en USD via FX spot (yfinance). "
    "Performances calculées sur clôtures fin de mois. "
    "Volatilité annualisée: std mensuelle 6M × √12. "
    "Scores momentum = somme des rangs inversés sur 1M/3M/6M/1Y/3Y.</i></font>",
    styles['Normal']
))

# Export
doc.build(story)
print(f"✅ Rapport généré : {OUTPUT_PDF}")


C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\3601286682.py:83: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(list(tickers.values()), start=start_date, progress=False)
C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\3601286682.py:92: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_prices = yf.download(fx_list, start=start_date, progress=False)["Close"] if fx_list else pd.DataFrame()
C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\3601286682.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rate = fx_prices[fx_map[region]].reindex(p.index).fillna(method="ffill")
C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\3601286682.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rate = fx_prices[fx_map[region]].reindex

✅ Rapport généré : Rapport_Momentum.pdf


In [39]:
# ============================================================
# CIO Market Snapshot (Equities USD) — V3 (Momentum + Risks + FX/Gold)
# ============================================================

import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
matplotlib.use("Agg")  # rendu off-screen
import matplotlib.pyplot as plt
from io import BytesIO
from datetime import datetime

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table
from reportlab.platypus import TableStyle
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

# ---------- Paramètres ----------
OUTPUT_PDF = "Rapport_Momentum.pdf"
LOOKBACK_YEARS = 4
HEATMAP_DPI = 180

# ---------- Univers Indices & FX (conversion en USD) ----------
tickers = {
    "US": "^GSPC",         # S&P 500 (USD)
    "Europe": "^STOXX50E", # EuroStoxx 50 (EUR)
    "Japan": "^N225",      # Nikkei 225 (JPY)
    "China": "^HSI",       # Hang Seng (HKD)
    "LatAm": "^BVSP"       # Bovespa (BRL)
}
fx_map = {
    "US": None,            # déjà USD
    "Europe": "EURUSD=X",
    "Japan": "JPY=X",      # USD/JPY
    "China": "HKD=X",      # USD/HKD
    "LatAm": "BRL=X"       # USD/BRL
}

# ---------- Styles ReportLab (tables finance) ----------
table_style = TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#2F4F4F")),
    ('TEXTCOLOR',(0,0),(-1,0),colors.white),
    ('ALIGN',(1,1),(-1,-1),'RIGHT'),
    ('ALIGN',(0,0),(0,-1),'LEFT'),
    ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
    ('FONTSIZE', (0,0), (-1,-1), 9),
    ('BOTTOMPADDING', (0,0), (-1,0), 6),
    ('BACKGROUND',(0,1),(-1,-1),colors.whitesmoke),
    ('GRID', (0,0), (-1,-1), 0.25, colors.grey)
])

# ---------- Utilitaires ----------
def month_end_prices(px: pd.DataFrame) -> pd.DataFrame:
    return px.resample("ME").last()   # ME = month end

def perf_point(df_monthly: pd.DataFrame, months: int) -> pd.Series:
    """Perf cumulée N mois ; renvoie NaN si pas assez d’historique"""
    if len(df_monthly) <= months:
        return pd.Series([np.nan]*df_monthly.shape[1], index=df_monthly.columns)
    return df_monthly.pct_change(periods=months).iloc[-1]

def make_perf_table(perfs: pd.DataFrame) -> list:
    header = [""] + list(perfs.columns)
    rows = []
    for idx in perfs.index:
        row = [idx] + [f"{100*perfs.loc[idx, col]:.2f}" for col in perfs.columns]
        rows.append(row)
    return [header] + rows

# ---------- Téléchargement données ----------
start_date = (pd.Timestamp.today().normalize() - pd.DateOffset(years=LOOKBACK_YEARS)).strftime("%Y-%m-%d")

# Prix indices (Close)
raw = yf.download(list(tickers.values()), start=start_date, progress=False)
prices = raw["Close"] if "Close" in raw else raw.copy()

# FX vers USD (Close)
fx_list = [fx for fx in fx_map.values() if fx is not None]
fx_prices = yf.download(fx_list, start=start_date, progress=False)["Close"] if fx_list else pd.DataFrame()

# ---------- Conversion en USD ----------
prices_usd = pd.DataFrame(index=prices.index)
for region, idx in tickers.items():
    p = prices[idx].copy()
    if fx_map[region] is None:
        prices_usd[region] = p
    else:
        rate = fx_prices[fx_map[region]].reindex(p.index).ffill()
        prices_usd[region] = p * rate

# ---------- Données fin de mois & Momentum ----------
monthly = month_end_prices(prices_usd).dropna(how="any")
horizons = {"1M":1, "3M":3, "6M":6, "1Y":12, "3Y":36}
if len(monthly) < (max(horizons.values()) + 1):
    raise ValueError("Pas assez d’historique pour 3Y. Augmente LOOKBACK_YEARS ou adapte les horizons.")

perfs = pd.DataFrame({h: perf_point(monthly, m) for h,m in horizons.items()})
perfs = perfs.reindex(columns=["1M","3M","6M","1Y","3Y"])
ranks = perfs.rank(ascending=False)  # 1 = meilleur

# ---------- Risques (vol 6/12M annualisée) & Corrélations ----------
monthly_returns = monthly.pct_change()
vol_6m = (monthly_returns.rolling(6).std().iloc[-1] * np.sqrt(12)).rename("Vol 6M %") * 100
vol_12m = (monthly_returns.rolling(12).std().iloc[-1] * np.sqrt(12)).rename("Vol 12M %") * 100
risk_table = pd.concat([vol_6m, vol_12m], axis=1).round(2)

corr_matrix = monthly_returns.corr()

# ---------- Heatmaps ----------
# Momentum ranks
ranks_int = ranks.astype(int)
plt.figure(figsize=(8,6))
sns.heatmap(ranks_int, annot=True, fmt="d",
            linewidths=0.5, linecolor="grey",
            cmap="RdYlGn_r", cbar=True,
            cbar_kws={'label': 'Rang (1 = Top performer)'},
            annot_kws={"size":12, "weight":"bold", "color":"black"})
plt.title("Momentum Ranking des Indices (en USD, fin de mois)", fontsize=14, weight="bold")
plt.tight_layout()
heatmap_buf = BytesIO(); plt.savefig(heatmap_buf, format="png", bbox_inches="tight", dpi=HEATMAP_DPI); plt.close(); heatmap_buf.seek(0)

# Corrélations
plt.figure(figsize=(6.5,5.5))
sns.heatmap(corr_matrix, annot=True, fmt=".2f",
            cmap="RdBu_r", center=0, linewidths=0.5, linecolor="grey")
plt.title("Corrélations des retours mensuels (3–4 ans)", fontsize=12, weight="bold")
plt.tight_layout()
corr_buf = BytesIO(); plt.savefig(corr_buf, format="png", bbox_inches="tight", dpi=150); plt.close(); corr_buf.seek(0)

# ---------- Graphiques base 100 (fin de mois) ----------
# YTD
ytd_start = pd.Timestamp(datetime.today().year, 1, 1)
monthly_ytd = monthly[monthly.index >= ytd_start].copy()
ytd_rebased = monthly_ytd.apply(lambda x: x / x.dropna().iloc[0] * 100)

plt.figure(figsize=(8,5))
for col in ytd_rebased.columns:
    plt.plot(ytd_rebased.index, ytd_rebased[col], label=col, linewidth=2)
plt.title("Indices mondiaux — YTD (Base 100, USD, fin de mois)", fontsize=14, weight="bold")
plt.ylabel("Base 100"); plt.xlabel("Date")
plt.legend(loc="upper left", fontsize=9); plt.grid(True, linestyle="--", alpha=0.6)
ytd_buf = BytesIO(); plt.savefig(ytd_buf, format="png", bbox_inches="tight", dpi=150); plt.close(); ytd_buf.seek(0)

# 3 ans
three_year_start = pd.Timestamp.today() - pd.DateOffset(years=3)
monthly_3y = monthly[monthly.index >= three_year_start].copy()
px_3y_rebased = monthly_3y.apply(lambda x: x / x.dropna().iloc[0] * 100)

plt.figure(figsize=(8,5))
for col in px_3y_rebased.columns:
    plt.plot(px_3y_rebased.index, px_3y_rebased[col], label=col, linewidth=2)
plt.title("Indices mondiaux — 3 ans (Base 100, USD, fin de mois)", fontsize=14, weight="bold")
plt.ylabel("Base 100"); plt.xlabel("Date")
plt.legend(loc="upper left", fontsize=9); plt.grid(True, linestyle="--", alpha=0.6)
three_buf = BytesIO(); plt.savefig(three_buf, format="png", bbox_inches="tight", dpi=150); plt.close(); three_buf.seek(0)

# ---------- FX & Gold snapshot ----------
fx_universe = {
    "EURUSD":"EURUSD=X",
    "USDJPY":"JPY=X",
    "USDCNY":"CNY=X",
    "USDBRL":"BRL=X",
    "Gold (GC=F)":"GC=F"
}

fx_raw = yf.download(list(fx_universe.values()), start=start_date, progress=False)["Close"]
fx_px = fx_raw.rename(columns={v:k for k,v in fx_universe.items()})
fx_monthly = month_end_prices(fx_px).dropna(how="all")  

fx_perfs = pd.DataFrame({h: perf_point(fx_monthly, m) for h,m in horizons.items()})
fx_perfs = fx_perfs.reindex(columns=["1M","3M","6M","1Y","3Y"]).round(4)

# ---------- PDF ----------
styles = getSampleStyleSheet()
doc = SimpleDocTemplate(OUTPUT_PDF, pagesize=A4)
story = []

# Titre + date
today_str = pd.Timestamp.today().strftime("%d %B %Y")
story.append(Paragraph("📌 CIO Market Snapshot — Momentum & Risks (USD)", styles['Title']))
story.append(Spacer(1, 6))
story.append(Paragraph(f"<i>Rapport généré le {today_str}</i>", styles['Normal']))
story.append(Spacer(1, 12))

# Commentaire court (optionnel)
leaders = ranks.mean(axis=1).sort_values().index[:2].tolist()
laggard = ranks.mean(axis=1).sort_values(ascending=False).index[0]
comment = f"Momentum: leaders = {leaders[0]}, {leaders[1]} ; laggard = {laggard}."
story.append(Paragraph(f"<b>Commentaire :</b> {comment}", styles['Normal']))
story.append(Spacer(1, 12))

# Heatmap Momentum
story.append(Paragraph("<b>Heatmap des Rangs Momentum</b>", styles['Heading3']))
story.append(Image(heatmap_buf, width=400, height=250))
story.append(Spacer(1, 12))

# Graphiques base 100
story.append(Paragraph("<b>Évolution YTD (Base 100, USD, fin de mois)</b>", styles['Heading3']))
story.append(Image(ytd_buf, width=400, height=250))
story.append(Spacer(1, 12))

story.append(Paragraph("<b>Évolution 3 ans (Base 100, USD, fin de mois)</b>", styles['Heading3']))
story.append(Image(three_buf, width=400, height=250))
story.append(Spacer(1, 12))

# Performances indices (USD)
story.append(Paragraph("<b>Performances des indices (%) — USD</b>", styles['Heading3']))
t_perf = Table(make_perf_table(perfs)); t_perf.setStyle(table_style); story.append(t_perf)
story.append(Spacer(1, 12))

# Volatilités
story.append(Paragraph("<b>Volatilité des indices (annualisée, %)</b>", styles['Heading3']))
risk_tbl = [["Région","Vol 6M %","Vol 12M %"]] + [[idx, f"{risk_table.loc[idx,'Vol 6M %']:.2f}", f"{risk_table.loc[idx,'Vol 12M %']:.2f}"] for idx in risk_table.index]
t_risk = Table(risk_tbl); t_risk.setStyle(table_style); story.append(t_risk)
story.append(Spacer(1, 12))

# Corrélations
story.append(Paragraph("<b>Corrélations entre indices (retours mensuels)</b>", styles['Heading3']))
story.append(Image(corr_buf, width=380, height=300))
story.append(Spacer(1, 12))

# FX & Or
story.append(Paragraph("<b>Marchés FX & Or — Performances (%)</b>", styles['Heading3']))
fx_tbl = [[""] + list(fx_perfs.columns)] + [[idx] + [f"{100*v:.2f}" for v in fx_perfs.loc[idx]] for idx in fx_perfs.index]
t_fx = Table(fx_tbl); t_fx.setStyle(table_style); story.append(t_fx)
story.append(Spacer(1, 12))

# Notes méthodo
story.append(Paragraph(
    "<font size=8><i>Notes: Données Yahoo Finance. Indices convertis en USD via FX spot; "
    "toutes les perfs/graphes sont en fin de mois. Vol annualisée: écart-type des retours mensuels × √12. "
    "Momentum: rangs par horizon (1M/3M/6M/1Y/3Y) sur perfs USD. Corrélations: retours mensuels. "
    "FX: EURUSD (EUR/USD), JPY=X (USD/JPY), etc.; DXY = Dollar Index, GC=F = Gold futures (USD).</i></font>",
    styles['Normal']
))

doc.build(story)
print(f"✅ Rapport généré : {OUTPUT_PDF}")


C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\1915814463.py:77: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(list(tickers.values()), start=start_date, progress=False)
C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\1915814463.py:82: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_prices = yf.download(fx_list, start=start_date, progress=False)["Close"] if fx_list else pd.DataFrame()
C:\Users\ilyes\AppData\Local\Temp\ipykernel_20476\1915814463.py:169: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_raw = yf.download(list(fx_universe.values()), start=start_date, progress=False)["Close"]


✅ Rapport généré : Rapport_Momentum.pdf
